In [7]:
import itertools
import numpy as np
import scipy.special as ss

In [12]:
p_vals = np.arange(1,13)
n_vals = np.arange(1,13)
e_vals = np.arange(33)

attack_length = 2
e_lists = list(itertools.product(*[e_vals]*attack_length))

# cartesian product of possible values of p, n, (e_{ell-1}, e_ell)
vectors = list(itertools.product(*[p_vals, n_vals, e_lists]))

In [13]:
def calcDelayDifference(p, n, e_list):
    e_ell = e_list[1]
    return 40 * (p - n) + 8 * max(24-e_ell, 0) - 8 * max(e_ell-8, 0)

def calcRewardDifference(p, e_list):
    block_reward = 16 * (1./(p+1) + e_list[1]/160. - 1./5)
    endorse_reward = 2 * e_list[0] * (1./(p+1) - 1)
    return block_reward + endorse_reward

def calcProb(alpha, p, n, e_list):
    binoms = [ss.binom(32, e) for e in e_list]
    return np.product(binoms) * np.power(alpha, n + sum(e_list) + 1) * np.power(1- alpha,65  + p - sum(e_list))

In [15]:
# minutes in a year
C = 365 * 24 * 60
for alpha in np.arange(0.1, 0.45, 0.05):
    probabilites = []
    values = []
    for p, n, e_list in vectors:
        reward = calcRewardDifference(p, e_list)
        # if feasible and profitable
        if (calcDelayDifference(p, n, e_list) < 0) and (reward > 0):
            prob = calcProb(alpha, p, n, e_list)
            probabilites.append(prob)
            values.append(prob * reward)
    # expected reward per minute is alpha * (block reward + total endorse reward)
#     expected_reward = alpha * ( 16 +  2 * 32 )
#     percent_increase = 100 * sum(values) / expected_reward 
    
    print(alpha, sum(probabilites) * C, sum(values) * C)

(0.1, 0.2194662791755539, 0.5373286410002961)
(0.15000000000000002, 4.1871727301842405, 7.938809127541264)
(0.20000000000000004, 28.594632144112392, 45.94477785186911)
(0.25000000000000006, 98.08980145954604, 130.5769505061823)
(0.30000000000000004, 188.46839048098573, 203.81957219266818)
(0.3500000000000001, 209.63311759736345, 190.1789114429282)
(0.40000000000000013, 136.95966839848606, 110.78632157045962)


In [5]:
# minutes in a year
C = 365 * 24 * 60
for alpha in np.arange(0.1, 0.45, 0.05):
    probabilites = []
    values = []
    for p, n, e_list in vectors:
        reward = calcRewardDifference(p, e_list)
        # if feasible and profitable
        if (calcDelayDifference(p, n, e_list) <= 0) and (reward > 0):
            prob = calcProb(alpha, p, n, e_list)
            probabilites.append(prob)
            values.append(prob * reward)
    # expected reward per minute is alpha * (block reward + total endorse reward)
    expected_reward = alpha * ( 16 +  2 * 32 )
    percent_increase = 100 * sum(values) / expected_reward 
    
    print(alpha, sum(probabilites) * C, sum(values) * C, percent_increase)

(0.1, 0, 0, 0.0)
(0.15000000000000002, 0, 0, 0.0)
(0.20000000000000004, 0, 0, 0.0)
(0.25000000000000006, 0, 0, 0.0)
(0.30000000000000004, 0, 0, 0.0)
(0.3500000000000001, 0, 0, 0.0)
(0.40000000000000013, 0, 0, 0.0)
